In [2]:
from MyLexer import MyLexer as Scanner

if __name__ == '__main__':
    lexer = Scanner()

    examples = [

    """A = zeros(5); # create 5x5 matrix filled with zeros
    B = ones(7);  # create 7x7 matrix filled with ones
    I = eye(10);  # create 10x10 matrix filled with ones on diagonal and zeros elsewhere
    D1 = A.+B' ;  # add element-wise A with transpose of B
    D2 -= A.-B' ; # substract element-wise A with transpose of B
    D3 *= A.*B' ; # multiply element-wise A with transpose of B
    D4 /= A./B' ; # divide element-wise A with transpose of B
    """
    ,

    """ res1 = 60.500;
    res2 = 60.;
    res3 = .500;
    res4 = 60.52E2;
    str = "Hello world";

    if (m==n) { 
        if (m >= n) 
            print res;
    }
    """,
    
    """ E1 = [ [ 1, 2, 3],
       [ 4, 5, 6],
       [ 7, 8, 9] ];    
    """,

    # own examples
    """ 4.0>$
    """,

    """ printer = 5.0
        print 1.0e-10
        print 5.
        print .53
        if (m >= 1)
        K = 1
        K / 1
    """,

    '''C = -A;     # assignemnt with unary expression
    C = B' ;    # assignemnt with matrix transpose
    C = A+B ;   # assignemnt with binary addition
    C = A-B ;   # assignemnt with binary substraction
    C = A*B ;   # assignemnt with binary multiplication
    C = A/B ;   # assignemnt with binary division
    C = A.+B ;  # add element-wise A to B
    C = A.-B ;  # substract B from A 
    C = A.*B ;  # multiply element-wise A with B
    C = A./B ;  # divide element-wise A by B

    C += B ;  # add B to C 
    C -= B ;  # substract B from C 
    C *= A ;  # multiply A with C
    C /= A ;  # divide A by C'''
    ]

    # for i, ex in enumerate(examples):
    i = 4
    ex = examples[i]
    print(f"Przykład {i+1}:\n")
    for tok in lexer.tokenize(ex):
        print(f"({tok.lineno}): {tok.type} ({tok.value})")
    if i+1!= len(examples): print("_____________________________")

Przykład 5:

(1): ID (printer)
(1): = (=)
(1): FLOAT (5.0)
(2): PRINT (print)
(2): FLOAT (1e-10)
(3): PRINT (print)
(3): FLOAT (5.0)
(4): PRINT (print)
(4): FLOAT (0.53)
(5): IF (if)
(5): ( (()
(5): ID (m)
(5): GE (>=)
(5): INTNUM (1)
(5): ) ())
(6): ID (K)
(6): = (=)
(6): INTNUM (1)
(7): ID (K)
(7): / (/)
(7): INTNUM (1)
_____________________________
